# Chapter 7 - Exercises

## 1.

In [185]:
import pandas as pd
import torch
import torch.nn as nn

df = pd.read_csv('chapter_07.csv')
df.head()

,SEX,AGE,PROMOTIONS,YEARS_EMPLOYED,DID_QUIT
0,0,25,2,3,0
1,0,30,2,3,0
2,0,26,2,3,0
3,0,25,1,2,0
4,0,28,1,2,0


In [186]:
batch_size = 10
learning_rate = 0.001
input_size = 4
hidden_size = 500
num_classes = 2
num_epochs = 200

target = torch.tensor(df['DID_QUIT'].values)
features = torch.tensor(df.drop('DID_QUIT', axis = 1).values, dtype=torch.float32)
train_dataset = torch.utils.data.TensorDataset(features, target)
test_dataset = torch.utils.data.TensorDataset(features, target)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class NeuralNet(nn.Module):

    def __init__(self, input_size, hidden_size, num_classes):

        super(NeuralNet, self).__init__()

        self.l1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)  

    def forward(self, input):

        output = self.l1(input)
        output = self.relu(output)
        output = self.l2(output)

        return output

model = NeuralNet(input_size, hidden_size, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for i, (features, targets) in enumerate(train_loader):
        outputs = model(features)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, loss: {loss.item():.4f}")


Epoch 10/200, loss: 0.4351
Epoch 20/200, loss: 0.1675
Epoch 30/200, loss: 3.4021
Epoch 40/200, loss: 18.0052
Epoch 50/200, loss: 5.5954
Epoch 60/200, loss: 0.5053
Epoch 70/200, loss: 2.4546
Epoch 80/200, loss: 3.1795
Epoch 90/200, loss: 0.1796
Epoch 100/200, loss: 4.1102
Epoch 110/200, loss: 0.0302
Epoch 120/200, loss: 2.8424
Epoch 130/200, loss: 3.1587
Epoch 140/200, loss: 0.4914
Epoch 150/200, loss: 0.0000
Epoch 160/200, loss: 2.1086
Epoch 170/200, loss: 1.2760
Epoch 180/200, loss: 0.0000
Epoch 190/200, loss: 0.4703
Epoch 200/200, loss: 0.8091


In [187]:
with torch.no_grad():
    n_correct = 0
    n_samples = len(test_loader.dataset)

    for features, targets in test_loader:
        outputs = model(features)
        _, predicted = torch.max(outputs, 1)
        n_correct += (predicted == targets).sum().item()

    print(f"Network accuracy: {100*n_correct/n_samples:.4f}%")

Network accuracy: 72.2222%
